# Import Site-Package

In [1]:
import random
import time,math
import numpy as np
import torch.nn as nn
import gymnasium as gym
import gymnasium.wrappers as gym_wrap
import matplotlib.pyplot as plt
import matplotlib.animation as animation #輸出動畫影片
from IPython import display
from tqdm import tqdm
import cv2

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
import collections
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Replay Buffer

In [3]:
class ReplayBuffer:
  def __init__(self,max_size=int(1e5), num_steps=1):
    self.s = np.zeros((max_size,4,72,72), dtype=np.float32)
    self.a = np.zeros((max_size,), dtype=np.int64)
    self.r = np.zeros((max_size, 1), dtype=np.float32)
    self.s_ = np.zeros((max_size,4,72,72), dtype=np.float32)
    self.done = np.zeros((max_size, 1), dtype=np.float32)
    self.info = np.zeros((max_size, 7), dtype=np.float32)
    self.ptr = 0
    self.size = 0
    self.max_size = max_size
    self.num_steps = num_steps

  def append(self,s,a,r,s_,done,info):
    self.s[self.ptr] = s
    self.a[self.ptr] = a
    self.r[self.ptr] = r
    self.s_[self.ptr] = s_
    self.done[self.ptr] = done
    self.info[self.ptr] = info
    self.ptr = (self.ptr + 1) % self.max_size
    self.size = min(self.size+1,self.max_size)

  def sample(self, batch_size):
    ind = np.random.randint(0, self.size, batch_size)
    s = torch.FloatTensor(self.s[ind]).to(device)
    s_ = torch.FloatTensor(self.s_[ind]).to(device)
    return s, torch.LongTensor(self.a[ind]).to(device), \
           torch.FloatTensor(self.r[ind]).to(device), \
           torch.FloatTensor(self.s_[ind]).to(device), \
           torch.FloatTensor(self.done[ind]).to(device), \
           torch.FloatTensor(self.info[ind]).to(device)

# VAE

### Encoder

In [4]:
class VAEEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, latent_size*2)

    def forward(self, x, latent_size = 50):
        x = self.fc2( torch.relu(self.fc1(x)) )
        mean, log_var = x.split(latent_size, dim=1)
        return mean, log_var

### Decoder

In [5]:
class VAEDecoder(nn.Module):
    def __init__(self, latent_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(latent_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.sigmoid( self.fc2( torch.relu(self.fc1(x)) ) )
        return x

### VAE Module

In [6]:
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super().__init__()
        self.encoder = VAEEncoder(input_size, hidden_size, latent_size)
        self.decoder = VAEDecoder(latent_size, hidden_size, input_size)

    def forward(self, x):
        mean, log_var = self.encoder(x)
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std).to(device)
        z = mean + std * eps    #random sampling
        reconstruct = self.decoder(z)
        return reconstruct, mean, log_var

### Loss Function

In [7]:
def VAE_Loss(reconstruct, x, mean, log_var):
    reconstruct_loss = nn.MSELoss(reduction='sum')(reconstruct, x)
    kl_loss = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    return reconstruct_loss + kl_loss

# Initialize

### load model

In [8]:
Load_File=0
Old_File=f"Model-{Load_File}.pt"
if Load_File>0:
  Log= np.load(f"Log-{Load_File}.npy", allow_pickle=True).item()
else:
  Log={"TrainReward":[],"TestReward":[],"Loss":[]}

In [9]:
VAE_model = VAE(input_size=5184, hidden_size=800, latent_size=50)
VAE_model.load_state_dict( torch.load("model_2500.pth") )
VAE_model.eval()

VAE(
  (encoder): VAEEncoder(
    (fc1): Linear(in_features=5184, out_features=800, bias=True)
    (fc2): Linear(in_features=800, out_features=100, bias=True)
  )
  (decoder): VAEDecoder(
    (fc1): Linear(in_features=50, out_features=800, bias=True)
    (fc2): Linear(in_features=800, out_features=5184, bias=True)
  )
)

### env

In [10]:
class ImageEnv(gym.Wrapper):
  def __init__(self, env, stack_frames=4, delay_op=50):
    super(ImageEnv, self).__init__(env)
    self.delay_op = delay_op
    self.stack_frames = stack_frames
  def reset(self):
    s, info = self.env.reset()
    for i in range(self.delay_op):
      s, r, terminated, truncated, info = self.env.step(0)
      s = cv2.cvtColor(s, cv2.COLOR_RGB2GRAY)
      s=s[:72, 12:84]/255.0 - 0.5
      self.stacked_state = np.tile( s , (self.stack_frames,1,1) )  # [4, 72, 72]
    return self.stacked_state, info

  def step(self, action):
    reward = 0
    for _ in range(self.stack_frames):
      s, r, terminated, truncated, info = self.env.step(action)
      if r==-100:terminated=True
      s = cv2.cvtColor(s, cv2.COLOR_RGB2GRAY)
      s=s[:72, 12:84]/255.0 - 0.5
      reward += r
      if terminated or truncated:break
      self.stacked_state = np.concatenate((self.stacked_state[1:], s[np.newaxis]), axis=0)
    return self.stacked_state, reward, terminated, truncated, info

In [11]:
env=gym.make('CarRacing-v3',render_mode="rgb_array",domain_randomize=False, continuous=False)
env = ImageEnv(env)

# DQN

In [12]:
class DQN(torch.nn.Module):
  def __init__(self, vae, n_act, latent_size = 50):
    super(DQN,self).__init__()
    self.vae = vae
    self.fc1 = torch.nn.Linear(latent_size+7, 256)
    self.fc2 = torch.nn.Linear(256, n_act)
  def forward(self, x, info, input_size = 5184):
    with torch.no_grad():
        mean, _ = self.vae.encoder(x[:, -1, :, :].view(x.shape[0], -1))
        info = torch.FloatTensor(info).reshape(7,-1)
        mean = torch.FloatTensor(mean).reshape(50,-1)
        x = torch.vstack((mean,info)).T
    x = torch.relu( self.fc1(x) )
    x = self.fc2(x)
    return x

# 搭建智能體Agent的類別

In [13]:
class DQNAgent():
  def __init__(self,gamma=0.9,eps_low=0.1,lr=0.00025):
    self.env = env
    self.n_act=self.env.action_space.n
    self.PredictDQN= DQN(VAE_model, self.n_act)
    self.TargetDQN= DQN(VAE_model, self.n_act)
    if Load_File>0:
      self.PredictDQN.load_state_dict(torch.load(Old_File))
      self.TargetDQN.load_state_dict(torch.load(Old_File))
    self.PredictDQN.to(device)
    self.TargetDQN.to(device)
    self.LossFun=torch.nn.SmoothL1Loss()
    self.optimizer=torch.optim.Adam(self.PredictDQN.parameters(),lr=lr)
    self.gamma=gamma
    self.eps_low=eps_low
    self.rb=ReplayBuffer(max_size=10000, num_steps=1)
  def change_info_type(self,_):
      with torch.no_grad():
          teml = [_["true_speed"],_["four_ABS_sensors"][0],_["four_ABS_sensors"][1],_["four_ABS_sensors"][2],_["four_ABS_sensors"][3],_["steering_wheel_position"],_["gyroscop"]]
          for i in range(7): teml[i] = np.float64(teml[i])
          info = np.array(teml)
          return info
  def PredictA(self,s,info):
    if s.ndim == 3:
        s = np.expand_dims(s, axis=0)  # 變成 (1, 4, 72, 72)
    with torch.no_grad():
        s_tensor = torch.FloatTensor(s).to(device)
        info_tensor = torch.FloatTensor(info).to(device)
        q_values = self.PredictDQN(s_tensor,info_tensor)
        return torch.argmax(q_values).item()
  def SelectA(self,a):
    return self.env.action_space.sample() if np.random.random()<self.EPS else a
  def Train(self,N_EPISODES):
    for i in tqdm(range(Load_File,N_EPISODES)):
      self.EPS=self.eps_low+(1-self.eps_low)*math.exp(-i*12/(N_EPISODES))
      total_reward=0
      s,info=self.env.reset()
      info = self.change_info_type(info)        
      while True:
        a=self.SelectA(self.PredictA(s,info))
        s_,r,done,stop,info=self.env.step(a)
        if done or stop: info = np.zeros(7,dtype=np.float64)
        else: info  = self.change_info_type(info)
        self.rb.append(s,a,r,s_,done,info)
        if self.rb.size > 200 and i%self.rb.num_steps==0:self.Learn()
        if i % 20==0:  self.TargetDQN.load_state_dict(self.PredictDQN.state_dict())
        s=s_
        total_reward+=r
        if done or stop:break
      # print(f"\n{total_reward}")
      Log["TrainReward"].append(total_reward)
      if i % 10 == 9:
        test_reward=self.Test()
        print(f"\n訓練次數{i+1}，總回報{test_reward}")
        Log["TestReward"].append(test_reward)
        torch.save(self.PredictDQN.state_dict(), f"Model-{i+1}.pt")
        np.save(f"Log-{i+1}.npy", Log)
  def Learn(self):
    self.optimizer.zero_grad()
    batch_s, batch_a, batch_r, batch_s_, batch_done, batch_info=self.rb.sample(32)
    predict_Q = (self.PredictDQN(batch_s.to(device),batch_info.to(device))*F.one_hot(batch_a.long().to(device),self.n_act)).sum(1,keepdims=True)
    with torch.no_grad():
      target_Q = batch_r.to(device)+(1-batch_done.to(device))*self.gamma*self.TargetDQN(batch_s_.to(device),batch_info.to(device)).max(1,keepdims=True)[0]
    loss = self.LossFun(predict_Q, target_Q)
    Log["Loss"].append(float(loss))
    loss.backward()
    self.optimizer.step()
  def Test(self,VIDEO=False):
    total_reward=0
    video=[]
    s,info=self.env.reset()
    info  = self.change_info_type(info)
    while True:
      video.append(self.env.render())
      a=self.PredictA(s,info)
      s,r,done,stop,info=self.env.step(a)
      if done or stop: info = np.zeros(7,dtype=np.float64)
      else: info  = self.change_info_type(info)
      total_reward+=r
      if done or stop:break
    if VIDEO:
      patch = plt.imshow(video[0]) #產生展示圖形物件
      plt.axis('off') #關閉坐標軸
      def animate(i): #設定更換影格的函數
        patch.set_data(video[i])
        #plt.gcf()=>建新繪圖區 animate=>更換影格函數 frames=>影格數 interval=>影隔間距(毫秒)
      anim = animation.FuncAnimation(plt.gcf(),animate,frames=len(video),interval=200)
      anim.save('Car_Racing.mp4') #儲存為mp4擋
    return total_reward
  def Record(self):
    total_reward=0
    s,_=self.env.reset()
    while True:
      image=self.env.render()
      plt.imshow(image)
      #plt.imsave(f"/content/drive/MyDrive/recording/{str(int(time.time()))}.png", image)
      a=self.PredictA(s)
      s,r,done,stop,_=self.env.step(a)
      print(r)
      total_reward+=r
      plt.pause(0.1)
      #清除目前的顯示
      display.clear_output(wait=True)
      if done or stop:break
    print(total_reward)

In [14]:
Agent=DQNAgent(gamma=0.95,eps_low=0.05,lr=0.00025)
Agent.Train(N_EPISODES=5000)

  0%|                                      | 10/5000 [02:23<18:51:47, 13.61s/it]


訓練次數10，總回報-2.9740614334471034


  0%|▏                                     | 20/5000 [04:48<20:11:38, 14.60s/it]


訓練次數20，總回報6.640579710145111


  1%|▏                                     | 30/5000 [06:31<14:01:57, 10.16s/it]

...


訓練次數4980，總回報38.28831168831151


100%|████████████████████████████████████▉| 4990/5000 [6:06:33<01:18,  7.82s/it]


訓練次數4990，總回報45.202090592334294


100%|█████████████████████████████████████| 5000/5000 [6:08:06<00:00,  4.42s/it]


訓練次數5000，總回報-94.99999999999903


In [16]:
# Agent.Record()